<a href="https://colab.research.google.com/github/Yuvasree9/Amica--Your-dearest-one/blob/main/Amica_2_O.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers faiss-cpu transformers duckduckgo-search


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
 

In [ ]:
!pip install duckduckgo-search


In [ ]:
from duckduckgo_search import DDGS


In [ ]:
def web_search(query):
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=3))
    if results:
        return results[0]["body"] if "body" in results[0] else results[0]["href"]
    else:
        return "Bro, net lo kuda information kanipinchaledu "


In [ ]:
print(web_search("Who is the current CEO of Google?"))


Pichai Sundararajan (born June 10, 1972), better known as Sundar Pichai (pronounced: / ˈ s ʊ n d ɜːr p ɪ ˈ tʃ eɪ /), is an American business executive. [3] [4] He is the chief executive officer (CEO) of Alphabet Inc. and its subsidiary Google.[5]Pichai began his career as a materials engineer.Following a short stint at the management consulting firm McKinsey & Co., Pichai joined Google ...


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Chat.json to Chat.json


In [ ]:
# Basic Imports
import json
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from duckduckgo_search import DDGS


# Web search function
def web_search(query):
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=3))
    if results:
        return results[0]["body"] if "body" in results[0] else results[0]["href"]
    else:
        return "Bro, net lo kuda information kanipinchaledu"


# Load chat messages
def load_messages(path):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    messages = [entry["message"] for entry in data if "message" in entry and entry["sender"] == "Sarfaraz Ahamad"]
    return messages
#sarfaraz Ahamad is my friend where he helped me to collect the customized dataset of friendly tone

# Embed messages and build FAISS index
def build_index(messages):
    encoder = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = encoder.encode(messages, show_progress_bar=True)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings))
    return encoder, index, embeddings

# Get similar responses
def retrieve_similar(query, encoder, index, messages, k=3):
    query_vec = encoder.encode([query])
    distances, indices = index.search(query_vec, k)
    return [messages[i] for i in indices[0]]

# Load open LLM
def load_llm():
    tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", use_auth_token=True)
    model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", device_map="auto")
    return pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)

# Final response generator
def generate_reply(query, encoder, index, messages, llm_pipeline):
    sims = retrieve_similar(query, encoder, index, messages)
    if sims and sims[0]:
        return f"Personal touch 🫂:\n{sims[0]}"
    else:
        # Web fallback
        web_info = web_search(query)
        if web_info:
            return f"Google se mila info 🌐:\n{web_info}"
        # LLM fallback
        response = llm_pipeline(f"Tu kya bolega jab koi puche: {query}")
        return "Soch ke bola bhai:\n" + response[0]['generated_text']

"Ready to load your file and build the pipeline!"


'Ready to load your file and build the pipeline!'

In [ ]:
import json

def load_messages(path="Chat.json"):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    messages = [d["message"] for d in data if "message" in d]
    return messages

messages = load_messages()
print("Sample:", messages[:4])


Sample: ['Bochulo photo tisi nak pettadaniki baleda?', 'Nak avasaram ledu anna', 'Inkot pettu', 'Ninnu avad adagala']


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

encoder = SentenceTransformer("all-MiniLM-L6-v2")
message_embeddings = encoder.encode(messages, show_progress_bar=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
import faiss

dimension = message_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(message_embeddings))


In [ ]:
def get_similar_messages(query, k=3):
    query_embedding = encoder.encode([query])
    distances, indices = index.search(query_embedding, k)
    return [messages[i] for i in indices[0]]


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "HuggingFaceH4/zephyr-7b-alpha"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

...

model-00004-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
from duckduckgo_search import DDGS

def web_search(query):
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=3))
    if results:
        return results[0]["body"] if "body" in results[0] else results[0]["href"]
    else:
        return "Bro, net lo kuda information kanipinchaledu"


In [ ]:
def rewrite_in_sarfaraz_style(answer):
    """Rewrites any factual or generated response in Sarfaraz-style slang."""
    style_prompt = f"""
    Convert the following answer into casual Hindi-English slang like a funny and emotionally supportive friend (Sarfaraz style):

    Original: {answer}
    Styled:
    """
    response = generator(style_prompt, max_new_tokens=60, do_sample=True, temperature=0.9)[0]["generated_text"]
    return response.split("Styled:")[-1].strip()


In [ ]:
def generate_response(user_input):
    similar = get_similar_messages(user_input)
    if similar:
        return f"Sarfaraz style lo cheppalante: {similar[0]}"

    # Web search fallback
    if any(q in user_input.lower() for q in ["what", "who", "when", "where", "how", "latest", "news"]):
        web_ans = web_search(user_input)
        return "Sarfaraz style lo cheppalante: " + rewrite_in_sarfaraz_style(web_ans)

    # Fallback to LLM and stylize
    prompt = f"Answer this in a factual way: {user_input}"
    raw_reply = generator(prompt, max_new_tokens=60, do_sample=True, temperature=0.7)[0]["generated_text"]
    return "Sarfaraz style lo cheppalante: " + rewrite_in_sarfaraz_style(raw_reply)



In [ ]:
print(generate_response("Why are you ignoring me? 😢"))



Sarfaraz style lo cheppalante: And I am mad about you 🙂
